In [3]:
import os
import re
import spacy
import nltk
import unicodedata
import pandas as pd
from nltk.corpus import stopwords
from dotenv import load_dotenv
from langchain_community.document_loaders import CSVLoader
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma
from util import dir_management

In [4]:
# %%

load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model="llama3-8b-8192", temperature=0.2)

In [5]:
# %%

def load_stopwords():
    try:
        return set(stopwords.words('portuguese'))
    except LookupError:
        nltk.download('stopwords')
        return set(stopwords.words('portuguese'))


stop_words = load_stopwords()

nlp = spacy.load('pt_core_news_sm')

csv_columns = ['product_name', 'site_category_lv1',
               'site_category_lv2', 'overall_rating', 'review_text']

In [6]:
# %%

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def remove_exclamations_and_periods(text):
    text = re.sub(r'[!.,@]', '', text)
    return text


def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word not in stop_words])


def remove_accents(text):
    text = unicodedata.normalize('NFD', text)
    text = re.sub(r'[\u0300-\u036f]', '', text)
    return text


def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)


def remove_filling_words(text):
    word_list = [
        'de', 'a', 'o', 'do', 'da', 'em', 'para', 'com', 'na', 'por',
        'uma', 'os', 'no', 'se', 'mas', 'as', 'dos', 'pois', 'né'
    ]
    return ' '.join([word for word in text.split() if word not in word_list])


def remove_repetitive_words(text):
    return re.sub(r'\b(\w+)( \1)+\b', '', text)


def remove_repetitive_letters(text):
    return re.sub(r'/(.)\1{3,}/g', '', text)


def batch_documents(documents, batch_size):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

In [7]:
df = pd.read_csv(os.path.join(
    dir_management.get_project_dir(), 'B2W-Reviews.csv'), nrows=5000)
df_reduced = df.drop(
    columns=[col for col in df.columns if col not in csv_columns])

for column in csv_columns:
    df_reduced[column] = df_reduced[column].apply(lambda x: clean_text(str(x)))
    df_reduced[column] = df_reduced[column].apply(
        lambda x: remove_exclamations_and_periods(str(x)))
    df_reduced[column] = df_reduced[column].apply(
        lambda x: remove_accents(str(x)))
    df_reduced[column] = df_reduced[column].apply(
        lambda x: remove_stop_words(str(x)))

df_reduced["review_text"] = df_reduced["review_text"].apply(
    lambda x: remove_filling_words(str(x)))
df_reduced["review_text"] = df_reduced["review_text"].apply(
    lambda x: remove_repetitive_words(str(x)))
df_reduced["review_text"] = df_reduced["review_text"].apply(
    lambda x: remove_repetitive_letters(str(x)))

result_file_name = f'B2W-Reviews-After-PLN.csv'
df_reduced.head(5000).sort_values('site_category_lv1').to_csv(
    os.path.join(dir_management.get_out_dir(), result_file_name), index= False)

# %%
# nao rodar esse (arquivo total do csv)
df = pd.read_csv(os.path.join(
    dir_management.get_project_dir(), 'B2W-Reviews.csv'))
df_reduced = df.drop(
    columns=[col for col in df.columns if col not in csv_columns])

for column in csv_columns:
    df_reduced[column] = df_reduced[column].apply(lambda x: clean_text(str(x)))
    df_reduced[column] = df_reduced[column].apply(
        lambda x: remove_exclamations_and_periods(str(x)))
    df_reduced[column] = df_reduced[column].apply(
        lambda x: remove_accents(str(x)))
    df_reduced[column] = df_reduced[column].apply(
        lambda x: remove_stop_words(str(x)))

df_reduced["review_text"] = df_reduced["review_text"].apply(
    lambda x: remove_filling_words(str(x)))
df_reduced["review_text"] = df_reduced["review_text"].apply(
    lambda x: remove_repetitive_words(str(x)))
df_reduced["review_text"] = df_reduced["review_text"].apply(
    lambda x: remove_repetitive_letters(str(x)))

result_file_name = f'B2W-Reviews-After-PLN.csv'
df_reduced.sort_values('site_category_lv1').to_csv(
    os.path.join(dir_management.get_out_dir(), result_file_name))

In [8]:
# %%

loader = CSVLoader(
    file_path=os.path.join(dir_management.get_out_dir(), result_file_name),
    encoding='utf-8',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
        'fieldnames': csv_columns
    }
)

docs = loader.load()

In [9]:
# %% 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

max_batch_size = 3000
vector_db_path = os.path.join(dir_management.get_project_dir(), 'chroma_db')
if  not os.path.exists(vector_db_path):
    vectorstore = Chroma(
        embedding_function=hf,
        collection_name='reviews',
        persist_directory=vector_db_path
    )

    for batch in batch_documents(splits, max_batch_size):
        vectorstore.add_texts(
            texts=[doc.page_content for doc in batch],
            metadatas=[doc.metadata for doc in batch]
        )

else:
    vectorstore = Chroma(
        embedding_function=hf,
        collection_name='reviews',
        persist_directory=vector_db_path
    )

retriever = vectorstore.as_retriever()

c:\Users\Douglas\Desktop\API-FATEC-6-SEM-IA\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Douglas\Desktop\API-FATEC-6-SEM-IA\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# %%

prompt_template = """

Responda sempre de forma clara e precisa em português do Brasil.
Você é um assistente especializado em marketing e feedback de clientes. Responda perguntas que estejam relacionadas a marketing, campanhas, review de clientes, avaliações de produtos ou publicidade. 
Para perguntas fora desse escopo, responda: "Essa pergunta está fora do escopo deste chatbot. Por favor, faça perguntas relacionadas a marketing."
Para perguntas sobre produtos, use apenas o feedback dos clientes fornecido no contexto para responder, não invente respostas. Se o contexto não tiver informações suficientes, responda: "Não há informações suficientes para responder a essa pergunta."

Contexto: {context}
Pergunta: {question}

Resposta:

"""

In [11]:
prompt_template2 = """
Responda sempre de forma clara e precisa em português do Brasil.


Contexto: {context}
Pergunta: {question}

Resposta:
"""

In [12]:
# usando o banco 

def custom_prompt(context, question):
    return prompt_template.format(context=context, question=question)

def run_rag_chain(question):

    retrieved_docs = retriever.invoke(question)  # aumentar para k=10 para ver o resultado 
    
    formatted_context = format_docs(retrieved_docs)

    full_prompt = custom_prompt(formatted_context, question)

    response = llm.invoke(full_prompt)

    parsed_response = StrOutputParser().parse(response)

    return parsed_response.content

In [13]:
# Sem usar o banco

def custom_prompt2(context, question):
    return prompt_template2.format(context=context, question=question)

# Função para testes com o LLM isolado
def run_llm_only_test(question):
    # sem acessar o banco de dados vetorial
    full_prompt = custom_prompt2("", question)  # Sem contexto apenas a pergunta
    response = llm.invoke(full_prompt)
    parsed_response = StrOutputParser().parse(response)
    return parsed_response.content

In [19]:
question = [
    'Como podemos melhorar a experiência do cliente com base nas avaliações recebidas?'
]

In [28]:

print("------------- testes com o banco de dados -------------")
response = run_rag_chain(question[0])
print(response)


------------- testes com o banco de dados -------------
Analisando as avaliações recebidas, é possível identificar alguns padrões e áreas de melhoria para melhorar a experiência do cliente. Aqui estão algumas sugestões:

1. **Melhoria na entrega e logística**: Houve casos de produtos não entregues ou entregues com atraso (cama maca massagem portatil diva tl-msg-13 trevalla e leitor codigo barras laser cabo usb 30cm datamax danfe boleto). É fundamental melhorar a logística e a comunicação com os clientes sobre o status da entrega.
2. **Atendimento ao cliente**: A falta de atendimento satisfatório e a demora em responder a solicitações de devolução ou assistência (leitor codigo barras laser cabo usb 30cm datamax danfe boleto) são problemas que precisam ser resolvidos.
3. **Confiança na marca**: A avaliação negativa do site Submarino (caixa som fm usb cartao sd edifier xm6pf 48w rms - bivolt) pode ser relacionada à falta de comprometimento com a qualidade e a sinceridade na comunicação co

In [29]:
print("------------- Testes com LLM Isolado (sem banco de dados) -------------")
response = run_llm_only_test(question[0])
print(response)

------------- Testes com LLM Isolado (sem banco de dados) -------------
Excelente pergunta!

Melhorar a experiência do cliente é um objetivo fundamental para qualquer empresa que deseja manter e aumentar sua base de clientes. Para isso, é fundamental analisar as avaliações recebidas e identificar áreas de melhoria. Aqui estão algumas dicas para melhorar a experiência do cliente com base nas avaliações:

1. **Analisar as avaliações**: Leia atentamente as avaliações recebidas e identifique os principais pontos positivos e negativos. Isso ajudará a entender melhor as necessidades e expectativas dos clientes.
2. **Identificar padrões**: Busque padrões em relação às críticas e elogios recebidos. Isso pode ajudar a identificar áreas específicas que precisam de melhoria.
3. **Desenvolver ações**: Baseado nas avaliações, desenvolva ações concretas para melhorar a experiência do cliente. Isso pode incluir treinamento para os funcionários, mudanças nos processos internos ou implementação de nova